# Use engineered features to train a model
This code will use the features created to train a logistic regression model.
The process is:
1. Test each feature individually to see their effectiveness.
2. Remove any features which corrolate highly with one another.
3. Order the features from most to least effective.
4. Test combinations of features by working down this list adding a new feature each time.
5. Use the combination with the highest TRP.

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score as auc_score

In [3]:
#Import and combine prices files
path = r"C:\Users\Robert\Documents\python_scripts\stock_trading_ml_modelling\historical_prices"
df_ft = pd.read_hdf(path + r"\all_hist_prices_w_ft_eng2.h5")
print("SHAPE: {}".format(df_ft.shape))
print(df_ft.dtypes)
df_ft.head()

SHAPE: (282789, 106)
ticker                                       object
date                                 datetime64[ns]
open                                        float64
close                                       float64
high                                        float64
                                          ...      
long_prev_max_move_date_macd_line           float64
long_prev_min_move_date_macd_line           float64
long_max_grad_macd_line                     float64
long_min_grad_macd_line                     float64
signal                                       object
Length: 106, dtype: object


,ticker,date,open,close,high,low,volume,change_price,per_change_price,ema26,...,min_change_macd_line,prev_max_grad_macd_line,prev_min_grad_macd_line,max_move_cum_macd_line,min_move_cum_macd_line,long_prev_max_move_date_macd_line,long_prev_min_move_date_macd_line,long_max_grad_macd_line,long_min_grad_macd_line,signal
214476,SBRY,1995-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,hold
214477,SBRY,1995-01-09,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,NaN,...,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,hold
214478,SBRY,1995-01-16,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,NaN,...,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,hold
214479,SBRY,1995-01-23,0.125278,1.000000,1.000000,0.599644,0.861339,0.874722,0.874722,NaN,...,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,sell
214480,SBRY,1995-01-30,1.000000,0.833416,0.861097,1.000000,0.546860,-0.166584,-0.199881,NaN,...,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,hold


In [3]:
df_ft.columns

Index(['Ticker', 'Date', 'Open', 'Close', 'High', 'Low', 'Volume',
       'change_price', 'per_change_price', 'close_shift1',
       'change_close_shift1', 'vol_shift1', 'change_vol_shift1',
       'EMA26_shift1', 'change_EMA26_shift1', 'prev_max_close',
       'prev_min_close', 'prev_max_EMA26', 'prev_min_EMA26',
       'prev_max_close_date_change', 'prev_min_close_date_change',
       'prev_max_MACD_date_change', 'prev_min_MACD_date_change',
       'prev_max_EMA26_date_change', 'prev_min_EMA26_date_change',
       'prev_max_signal_date_change', 'prev_min_signal_date_change',
       'close_13_norm', 'close_26_norm', 'close_52_norm', 'MACD_pos_val',
       'MACD_neg_val', 'signal_pos_val', 'signal_neg_val',
       'change_price_pos_val', 'change_price_neg_val',
       'per_change_price_pos_val', 'per_change_price_neg_val',
       'max_change_close_pos_val', 'max_change_close_neg_val',
       'min_change_close_pos_val', 'min_change_close_neg_val',
       'max_change_MACD_pos_val', 'max_

# Build the logistic regression model
This model is designed to predict if a week should be buy, hold or sell.


In [4]:
#Import the modules
from sklearn.linear_model import LogisticRegression as lr

In [5]:
feature_cols = [
    #NORMALISED COLS
    #Standard features
    "open"
    ,"close"
    ,"high"
    ,"low"
    ,"volume"
    ,"change_price"
    ,"per_change_price"
    #Shifted features
    ,"close_shift1"
    ,"change_close_shift1"
    ,"vol_shift1"
    ,"change_vol_shift1"
    ,"ema26_shift1"
    ,"change_ema26_shift1"
    #change to periodic max mins
    ,"close_13_norm"
    ,"close_26_norm"
    ,"close_52_norm"
    ,"macd_line_13_norm"
    ,"macd_line_26_norm"
    ,"macd_line_52_norm"
    #Pos neg features
    ,"macd_pos_val"
    ,"macd_neg_val"
    ,"signal_line_pos_val"
    ,"signal_line_neg_val"
    ,"change_price_pos_val"
    ,"change_price_neg_val"
    ,"per_change_price_pos_val"
    ,"per_change_price_neg_val" 
]
#Append additional columns for key areas
for col in ['close','macd','ema26','signal_line','macd_line']:
    #Prev max/min features
    feature_cols.append("prev_max_{}".format(col))
    feature_cols.append("prev_min_{}".format(col))
    #date changes
    feature_cols.append("prev_max_{}_date_change".format(col))
    feature_cols.append("prev_min_{}_date_change".format(col))
    #Min max change features
    feature_cols.append("max_change_{}_pos_val".format(col))
    feature_cols.append("max_change_{}_neg_val".format(col))
    feature_cols.append("min_change_{}_pos_val".format(col))
    feature_cols.append("min_change_{}_neg_val".format(col))
cat_cols = [
    'signal'
]
df_model = df_ft[feature_cols+cat_cols].copy()
print("COLUMNS:",df_model.columns)
print("COLUMNS:",df_model.dtypes)
print("SHAPE:",df_model.shape)

COLUMNS: Index(['Open', 'Close', 'High', 'Low', 'Volume', 'change_price',
       'per_change_price', 'close_shift1', 'change_close_shift1', 'vol_shift1',
       'change_vol_shift1', 'EMA26_shift1', 'change_EMA26_shift1',
       'prev_max_close', 'prev_min_close', 'prev_max_EMA26', 'prev_min_EMA26',
       'prev_max_close_date_change', 'prev_min_close_date_change',
       'prev_max_MACD_date_change', 'prev_min_MACD_date_change',
       'prev_max_EMA26_date_change', 'prev_min_EMA26_date_change',
       'prev_max_signal_date_change', 'prev_min_signal_date_change',
       'close_13_norm', 'close_26_norm', 'close_52_norm', 'MACD_pos_val',
       'MACD_neg_val', 'signal_pos_val', 'signal_neg_val',
       'change_price_pos_val', 'change_price_neg_val',
       'per_change_price_pos_val', 'per_change_price_neg_val',
       'max_change_close_pos_val', 'max_change_close_neg_val',
       'min_change_close_pos_val', 'min_change_close_neg_val',
       'max_change_MACD_pos_val', 'max_change_MACD_neg_

In [6]:
#Remove rows with missing or infinate values
df_model.dropna(inplace=True)
df_model.reset_index(inplace=True)
print("COLUMNS:",df_model.columns)
print("SHAPE:",df_model.shape)

COLUMNS: Index(['index', 'Open', 'Close', 'High', 'Low', 'Volume', 'change_price',
       'per_change_price', 'close_shift1', 'change_close_shift1', 'vol_shift1',
       'change_vol_shift1', 'EMA26_shift1', 'change_EMA26_shift1',
       'prev_max_close', 'prev_min_close', 'prev_max_EMA26', 'prev_min_EMA26',
       'prev_max_close_date_change', 'prev_min_close_date_change',
       'prev_max_MACD_date_change', 'prev_min_MACD_date_change',
       'prev_max_EMA26_date_change', 'prev_min_EMA26_date_change',
       'prev_max_signal_date_change', 'prev_min_signal_date_change',
       'close_13_norm', 'close_26_norm', 'close_52_norm', 'MACD_pos_val',
       'MACD_neg_val', 'signal_pos_val', 'signal_neg_val',
       'change_price_pos_val', 'change_price_neg_val',
       'per_change_price_pos_val', 'per_change_price_neg_val',
       'max_change_close_pos_val', 'max_change_close_neg_val',
       'min_change_close_pos_val', 'min_change_close_neg_val',
       'max_change_MACD_pos_val', 'max_change_

In [7]:
#Shuffle the dataset
np.random.seed(0)
rand_index = np.random.permutation(df_model.index.values)
df_model_rand = df_model.iloc[rand_index]
df_model_rand.index.values

array([100387,   5002, 119346, ..., 173685,  43567, 199340], dtype=int64)

In [8]:
#Create the train and test dataset
ind_lim = int(np.round(len(df_model_rand.index)*0.7))
print("ind_lim: {}".format(ind_lim))

df_train = df_model_rand.iloc[:ind_lim].reset_index()
df_test = df_model_rand.iloc[ind_lim:].reset_index()

print("train len: {}".format(len(df_train)))
print("test len: {}".format(len(df_test)))

ind_lim: 140603
train len: 140603
test len: 60259


In [9]:
#Function for building a model and outputting a dictionary of models created
def build_models(unique_classes,df_train,features):
    #Create a dictionary of models
    models = {}
    for cl in unique_classes:
        lm = lr() #create the model
        x_train = df_train[features]
        y_train = df_train["signal"] == cl
        lm.fit(x_train,y_train)
        models[cl] = lm
    return models

#Function for creating a dataframe with model probabilities and the most likely outcome
def calc_probs_df(models,unique_classes,df_test,features):
    modelled_probs = pd.DataFrame(columns=unique_classes)
    for cl in unique_classes:
        x_test = df_test[features]
        modelled_probs[cl] = models[cl].predict_proba(x_test)[:,1] 
        #Column index 1 as predict_proba outputs 2 columns,
            #the first is the probability that it is a negative result
            #the second is the probability that it is a positive result
            #that's why we want column index 1, we are looking at the positive result
    #Determine the most probable result
    modelled_probs["signal"] = modelled_probs.idxmax(axis=1)
    modelled_probs["signal_prob"] = modelled_probs[unique_classes].max(axis=1)
#     modelled_probs["bs_bool"] = (modelled_probs["buy"] >= 1/3) | (modelled_probs["sell"] >= 1/3)
#     modelled_probs.loc[modelled_probs["bs_bool"],"signal"] = modelled_probs[["buy","sell"]].idxmax(axis=1)
#     modelled_probs.loc[modelled_probs["bs_bool"],"signal_prob"] = modelled_probs[["buy","sell"]].max(axis=1)
#     print(modelled_probs)
    for cl in unique_classes:
        print(cl,"count:",str(len(modelled_probs.loc[modelled_probs["signal"] == cl,:])))
    return modelled_probs[["signal","signal_prob"]]

#Function to calculate the True Positive Rate for each classifier
def calc_tpr(pred_signal_s,real_signal_s,features,unique_classes):
    tpr_li = []
    for cl in unique_classes:
        true_pos = ((real_signal_s == cl) & (pred_signal_s == cl)).sum()
        false_pos = ((real_signal_s != cl) & (pred_signal_s == cl)).sum()
        true_neg = ((real_signal_s != cl) & (pred_signal_s != cl)).sum()
        false_neg = ((real_signal_s == cl) & (pred_signal_s != cl)).sum()
        #Calc tpr
        if sum([true_pos,false_neg]) != 0:
            tpr = true_pos / sum([true_pos,false_neg])
        else:
            tpr = 0
        #calc fpr
        if sum([true_pos,false_neg]) != 0:
            fpr = false_pos / sum([false_pos,true_neg])
        else:
            fpr = 0
        #calc accuracy
        if sum([true_pos,false_pos,true_neg,false_neg]) != 0:
            acc = sum([true_pos,true_neg])/sum([true_pos,false_pos,true_neg,false_neg])
        else:
            acc = 0
        if sum([true_pos,false_pos]) != 0:
            ppv = true_pos/sum([true_pos,false_pos])
        else:
            ppv = 0
        if sum([true_neg,false_neg]) != 0:
            npv = true_neg/sum([true_neg,false_neg])
        else:
            npv = 0
        #Calc the auc
        bool_pred_s = pred_signal_s == cl
        bool_real_s = real_signal_s == cl
        auc = auc_score(bool_real_s,bool_pred_s)
        tpr_li.append({
            "features":features.copy()
            ,"feature_count":len(features)
            ,"signal":cl
            ,"true_pos":true_pos
            ,"false_pos":false_pos
            ,"true_neg":true_neg
            ,"false_neg":false_neg
            ,"tpr":tpr
            ,"fpr":fpr
            ,"acc":acc
            ,"ppv":ppv
            ,"npv":npv
            ,"auc":auc
        })
        print('{} -> true_pos:{:.4f}, false_pos:{}, acc:{:.4f},ppv:{:.4f},npv:{:.4f},auc:{:.4f}'.format(cl,true_pos,false_pos,acc,ppv,npv,auc))
    tpr_df = pd.DataFrame(tpr_li,columns=["features","feature_count","signal","true_pos","false_pos","true_neg","false_neg","tpr","fpr","acc","ppv","npv","auc"])
    return tpr_df

# Create a base level model
For "buy", "sell" and "hold" test what the accuracy is if all signals are positive.

In [10]:
#Buy model
unique_classes = df_model_rand["signal"].unique()
calc_tpr(np.full((1,df_test.shape[0]),"buy")[0],df_test["signal"],["ALL BUY"],unique_classes)

buy -> true_pos:5023.0000, false_pos:55236, acc:0.0834,ppv:0.0834,npv:0.0000,auc:0.5000
hold -> true_pos:0.0000, false_pos:0, acc:0.4148,ppv:0.0000,npv:0.4148,auc:0.5000
sell -> true_pos:0.0000, false_pos:0, acc:0.6685,ppv:0.0000,npv:0.6685,auc:0.5000


,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,ppv,npv,auc
0,[ALL BUY],1,buy,5023,55236,0,0,1.0,1.0,0.083357,0.083357,0.000000,0.5
1,[ALL BUY],1,hold,0,0,24997,35262,0.0,0.0,0.414826,0.000000,0.414826,0.5
2,[ALL BUY],1,sell,0,0,40285,19974,0.0,0.0,0.668531,0.000000,0.668531,0.5


In [11]:
#Sell model
calc_tpr(np.full((1,df_test.shape[0]),"sell")[0],df_test["signal"],["ALL SELL"],unique_classes)

buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:0.0000, false_pos:0, acc:0.4148,ppv:0.0000,npv:0.4148,auc:0.5000
sell -> true_pos:19974.0000, false_pos:40285, acc:0.3315,ppv:0.3315,npv:0.0000,auc:0.5000


,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,ppv,npv,auc
0,[ALL SELL],1,buy,0,0,55236,5023,0.0,0.0,0.916643,0.000000,0.916643,0.5
1,[ALL SELL],1,hold,0,0,24997,35262,0.0,0.0,0.414826,0.000000,0.414826,0.5
2,[ALL SELL],1,sell,19974,40285,0,0,1.0,1.0,0.331469,0.331469,0.000000,0.5


In [12]:
#Buy model
calc_tpr(np.full((1,df_test.shape[0]),"hold")[0],df_test["signal"],["ALL HOLD"],unique_classes)

buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:35262.0000, false_pos:24997, acc:0.5852,ppv:0.5852,npv:0.0000,auc:0.5000
sell -> true_pos:0.0000, false_pos:0, acc:0.6685,ppv:0.0000,npv:0.6685,auc:0.5000


,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,ppv,npv,auc
0,[ALL HOLD],1,buy,0,0,55236,5023,0.0,0.0,0.916643,0.000000,0.916643,0.5
1,[ALL HOLD],1,hold,35262,24997,0,0,1.0,1.0,0.585174,0.585174,0.000000,0.5
2,[ALL HOLD],1,sell,0,0,40285,19974,0.0,0.0,0.668531,0.000000,0.668531,0.5


# Creating a multi-classification model
The model will take a one-vs-all approach (IE 1 if it is this value, 0 if it is anything else) using the variables of "buy", "hold", and "sell" individually and building a model to find the error rate on each one.

Error rate is determined by the four classifications:
- True positive - correct - model is 1, actual is 1
- True negative - correct - model is 0, actual is 0
- False positive - error - model is 1, actual is 0
- False negative - error - model is 0, actual is 1

# Comparing individual features
- Loop through the feaures to find the tpr of each one.
- Then eliminate features with a very low accuracy.
- Then order in descending order of combined buy,sell,hold accuracy and combine in that order to find the best feature combination.

In [13]:
tpr_single_df = pd.DataFrame([])
count = 0
for ft in feature_cols:
    print("")
    print("count: {}".format(count))
    count += 1
    print("TESTING:",ft)
    #Create a list of unique values
    unique_classes = df_model_rand["signal"].unique()
#     print("unique_classes:",unique_classes)
    #Remove rows with nan or inf
    tmp_df_train = df_train[[ft,"signal"]].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    tmp_df_test = df_test[[ft,"signal"]].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    #Build each model and add to the models dictionary
    models = build_models(unique_classes,tmp_df_train,[ft])
#     print(models)
    #Calculate the probabilities of each
    modelled_probs = calc_probs_df(models,unique_classes,tmp_df_test,[ft])
#     print("modelled_probs: {}".format(modelled_probs.head()))
    #Compare these results to the actual results
    modelled_probs["real_signal"] = tmp_df_test["signal"]
    #Calculate the True Positive Rate (TPR) for each of the unique classifiers
    tpr_single_df = tpr_single_df.append(calc_tpr(modelled_probs["signal"],modelled_probs["real_signal"],[ft],unique_classes))
tpr_single_df


count: 0
TESTING: Open
buy count: 0
hold count: 56025
sell count: 4234
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33726.0000, false_pos:22299, acc:0.6045,ppv:0.6020,npv:0.6372,auc:0.5322
sell -> true_pos:1800.0000, false_pos:2434, acc:0.6580,ppv:0.4251,npv:0.6756,auc:0.5148

count: 1
TESTING: Close
buy count: 0
hold count: 54442
sell count: 5817
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33063.0000, false_pos:21379, acc:0.6087,ppv:0.6073,npv:0.6220,auc:0.5412
sell -> true_pos:2445.0000, false_pos:3372, acc:0.6531,ppv:0.4203,npv:0.6780,auc:0.5194

count: 2
TESTING: High
buy count: 0
hold count: 56875
sell count: 3384
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:34120.0000, false_pos:22755, acc:0.6034,ppv:0.5999,npv:0.6625,auc:0.5287
sell -> true_pos:1500.0000, false_pos:1884, acc:0.6622,ppv:0.4433,npv:0.6752,auc:0.5142

count

hold -> true_pos:33987.0000, false_pos:23281, acc:0.5925,ppv:0.5935,npv:0.5737,auc:0.5162
sell -> true_pos:1327.0000, false_pos:1664, acc:0.6629,ppv:0.4437,npv:0.6744,auc:0.5126

count: 25
TESTING: close_13_norm
buy count: 0
hold count: 42454
sell count: 17805
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:29678.0000, false_pos:12776, acc:0.6953,ppv:0.6991,npv:0.6864,auc:0.6653
sell -> true_pos:8466.0000, false_pos:9339, acc:0.6540,ppv:0.4755,npv:0.7289,auc:0.5960

count: 26
TESTING: close_26_norm
buy count: 0
hold count: 43332
sell count: 16927
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:30099.0000, false_pos:13233, acc:0.6947,ppv:0.6946,npv:0.6950,auc:0.6621
sell -> true_pos:8250.0000, false_pos:8677, acc:0.6614,ppv:0.4874,npv:0.7294,auc:0.5988

count: 27
TESTING: close_52_norm
buy count: 0
hold count: 44946
sell count: 15313
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0

buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:35262.0000, false_pos:24997, acc:0.5852,ppv:0.5852,npv:0.0000,auc:0.5000
sell -> true_pos:0.0000, false_pos:0, acc:0.6685,ppv:0.0000,npv:0.6685,auc:0.5000

count: 49
TESTING: max_change_signal_neg_val
buy count: 0
hold count: 60259
sell count: 0
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:35262.0000, false_pos:24997, acc:0.5852,ppv:0.5852,npv:0.0000,auc:0.5000
sell -> true_pos:0.0000, false_pos:0, acc:0.6685,ppv:0.0000,npv:0.6685,auc:0.5000

count: 50
TESTING: min_change_signal_pos_val
buy count: 0
hold count: 60259
sell count: 0
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:35262.0000, false_pos:24997, acc:0.5852,ppv:0.5852,npv:0.0000,auc:0.5000
sell -> true_pos:0.0000, false_pos:0, acc:0.6685,ppv:0.0000,npv:0.6685,auc:0.5000

count: 51
TESTING: min_change_signal_neg_val
buy count: 0


,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,ppv,npv,auc
0,[Open],1,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000
1,[Open],1,hold,33726,22299,2698,1536,0.956440,0.892067,0.604457,0.601981,0.637222,0.532187
2,[Open],1,sell,1800,2434,37851,18174,0.090117,0.060420,0.658010,0.425130,0.675609,0.514849
0,[Close],1,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000
1,[Close],1,hold,33063,21379,3618,2199,0.937638,0.855263,0.608722,0.607307,0.621970,0.541188
2,[Close],1,sell,2445,3372,36913,17529,0.122409,0.083704,0.653147,0.420320,0.678024,0.519353
0,[High],1,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000
1,[High],1,hold,34120,22755,2242,1142,0.967614,0.910309,0.603429,0.599912,0.662530,0.528652
2,[High],1,sell,1500,1884,38401,18474,0.075098,0.046767,0.662158,0.443262,0.675182,0.514165
0,[Low],1,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000


In [14]:
# tpr_single_df.loc[tpr_single_df["signal"] == "buy",:]
tpr_single_df.loc[(tpr_single_df["signal"] == "buy") & (tpr_single_df["true_pos"] > 0),:].sort_values("auc",ascending=False)

,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,ppv,npv,auc
0,[prev_max_close_date_change],1,buy,3,5,55231,5020,0.000597,0.000091,0.91661,0.375,0.916682,0.500253


# Paired feature testing
Check all the pairings of features to see if this provides as better outcome.

In [15]:
#Create pairings
ft_li = []
for ft1 in feature_cols:
    for ft2 in feature_cols:
        #Skip if identical
        if ft1 == ft2:
            continue
        if ft1 > ft2:
            if [ft1,ft2] not in ft_li:
                ft_li.append([ft1,ft2])
        else:
            if [ft2,ft1] not in ft_li:
                ft_li.append([ft2,ft1])
print(len(ft_li))
ft_li

1326


[['Open', 'Close'],
 ['Open', 'High'],
 ['Open', 'Low'],
 ['Volume', 'Open'],
 ['change_price', 'Open'],
 ['per_change_price', 'Open'],
 ['close_shift1', 'Open'],
 ['change_close_shift1', 'Open'],
 ['vol_shift1', 'Open'],
 ['change_vol_shift1', 'Open'],
 ['Open', 'EMA26_shift1'],
 ['change_EMA26_shift1', 'Open'],
 ['prev_max_close', 'Open'],
 ['prev_min_close', 'Open'],
 ['prev_max_EMA26', 'Open'],
 ['prev_min_EMA26', 'Open'],
 ['prev_max_close_date_change', 'Open'],
 ['prev_min_close_date_change', 'Open'],
 ['prev_max_MACD_date_change', 'Open'],
 ['prev_min_MACD_date_change', 'Open'],
 ['prev_max_EMA26_date_change', 'Open'],
 ['prev_min_EMA26_date_change', 'Open'],
 ['prev_max_signal_date_change', 'Open'],
 ['prev_min_signal_date_change', 'Open'],
 ['close_13_norm', 'Open'],
 ['close_26_norm', 'Open'],
 ['close_52_norm', 'Open'],
 ['Open', 'MACD_pos_val'],
 ['Open', 'MACD_neg_val'],
 ['signal_pos_val', 'Open'],
 ['signal_neg_val', 'Open'],
 ['change_price_pos_val', 'Open'],
 ['change_

In [16]:
#Create pairings
tpr_pair_df = pd.DataFrame([])
count = 0
print("TOTAL TO RUN: {}".format(len(ft_li)))
for ft_pair in ft_li:
    print("")
    print("count: {}".format(count))
    count += 1
    print("TESTING: {},{}".format(ft_pair[0],ft_pair[1]))
    #Create a list of unique values
    unique_classes = df_model_rand["signal"].unique()
#     print("unique_classes:",unique_classes)
    #Remove rows with nan or inf
    tmp_df_train = df_train[[ft_pair[0],ft_pair[1],"signal"]].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    tmp_df_test = df_test[[ft_pair[0],ft_pair[1],"signal"]].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    #Build each model and add to the models dictionary
    models = build_models(unique_classes,tmp_df_train,[ft_pair[0],ft_pair[1]])
#     print(models)
    #Calculate the probabilities of each
    modelled_probs = calc_probs_df(models,unique_classes,tmp_df_test,[ft_pair[0],ft_pair[1]])
#     print("modelled_probs: {}".format(modelled_probs.head()))
    #Compare these results to the actual results
    modelled_probs["real_signal"] = tmp_df_test["signal"]
    #Calculate the True Positive Rate (TPR) for each of the unique classifiers
    tpr_pair_df = tpr_pair_df.append(calc_tpr(modelled_probs["signal"],modelled_probs["real_signal"],[ft_pair[0],ft_pair[1]],unique_classes))
tpr_pair_df

TOTAL TO RUN: 1326

count: 0
TESTING: Open,Close
buy count: 10
hold count: 53713
sell count: 6536
buy -> true_pos:3.0000, false_pos:7, acc:0.9166,ppv:0.3000,npv:0.9167,auc:0.5002
hold -> true_pos:33011.0000, false_pos:20702, acc:0.6191,ppv:0.6146,npv:0.6561,auc:0.5540
sell -> true_pos:2896.0000, false_pos:3640, acc:0.6562,ppv:0.4431,npv:0.6821,auc:0.5273

count: 1
TESTING: Open,High
buy count: 0
hold count: 56063
sell count: 4196
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33763.0000, false_pos:22300, acc:0.6051,ppv:0.6022,npv:0.6428,auc:0.5327
sell -> true_pos:1791.0000, false_pos:2405, acc:0.6583,ppv:0.4268,npv:0.6757,auc:0.5150

count: 2
TESTING: Open,Low
buy count: 38
hold count: 53901
sell count: 6320
buy -> true_pos:6.0000, false_pos:32, acc:0.9162,ppv:0.1579,npv:0.9167,auc:0.5003
hold -> true_pos:32984.0000, false_pos:20917, acc:0.6151,ppv:0.6119,npv:0.6417,auc:0.5493
sell -> true_pos:2684.0000, false_pos:3636, acc:0.6527,ppv

hold -> true_pos:32867.0000, false_pos:21080, acc:0.6104,ppv:0.6092,npv:0.6206,auc:0.5444
sell -> true_pos:2762.0000, false_pos:3550, acc:0.6555,ppv:0.4376,npv:0.6809,auc:0.5251

count: 24
TESTING: close_13_norm,Open
buy count: 0
hold count: 42419
sell count: 17840
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:29843.0000, false_pos:12576, acc:0.7014,ppv:0.7035,npv:0.6962,auc:0.6716
sell -> true_pos:8538.0000, false_pos:9302, acc:0.6559,ppv:0.4786,npv:0.7304,auc:0.5983

count: 25
TESTING: close_26_norm,Open
buy count: 0
hold count: 43361
sell count: 16898
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:30140.0000, false_pos:13221, acc:0.6956,ppv:0.6951,npv:0.6969,auc:0.6629
sell -> true_pos:8212.0000, false_pos:8686, acc:0.6607,ppv:0.4860,npv:0.7287,auc:0.5978

count: 26
TESTING: close_52_norm,Open
buy count: 0
hold count: 44957
sell count: 15302
buy -> true_pos:0.0000, false_pos:0, acc

buy count: 0
hold count: 53905
sell count: 6354
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:32854.0000, false_pos:21051, acc:0.6107,ppv:0.6095,npv:0.6210,auc:0.5448
sell -> true_pos:2742.0000, false_pos:3612, acc:0.6541,ppv:0.4315,npv:0.6803,auc:0.5238

count: 48
TESTING: max_change_signal_neg_val,Open
buy count: 0
hold count: 55884
sell count: 4375
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33909.0000, false_pos:21975, acc:0.6129,ppv:0.6068,npv:0.6907,auc:0.5413
sell -> true_pos:2120.0000, false_pos:2255, acc:0.6663,ppv:0.4846,npv:0.6805,auc:0.5251

count: 49
TESTING: min_change_signal_pos_val,Open
buy count: 0
hold count: 55426
sell count: 4833
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33377.0000, false_pos:22049, acc:0.6028,ppv:0.6022,npv:0.6100,auc:0.5322
sell -> true_pos:2030.0000, false_pos:2803, acc:0.6557,ppv:0.4200

hold -> true_pos:32699.0000, false_pos:20844, acc:0.6116,ppv:0.6107,npv:0.6184,auc:0.5467
sell -> true_pos:2894.0000, false_pos:3822, acc:0.6531,ppv:0.4309,npv:0.6810,auc:0.5250

count: 71
TESTING: prev_min_EMA26_date_change,Close
buy count: 0
hold count: 53866
sell count: 6393
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33232.0000, false_pos:20634, acc:0.6239,ppv:0.6169,npv:0.6825,auc:0.5585
sell -> true_pos:3092.0000, false_pos:3301, acc:0.6651,ppv:0.4837,npv:0.6866,auc:0.5364

count: 72
TESTING: prev_max_signal_date_change,Close
buy count: 0
hold count: 52182
sell count: 8077
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:32298.0000, false_pos:19884, acc:0.6208,ppv:0.6189,npv:0.6330,auc:0.5602
sell -> true_pos:3574.0000, false_pos:4503, acc:0.6531,ppv:0.4425,npv:0.6857,auc:0.5336

count: 73
TESTING: prev_min_signal_date_change,Close
buy count: 0
hold count: 53303
sell count: 6956

sell -> true_pos:2313.0000, false_pos:3029, acc:0.6566,ppv:0.4330,npv:0.6784,auc:0.5203

count: 94
TESTING: max_change_EMA26_neg_val,Close
buy count: 0
hold count: 55012
sell count: 5247
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33619.0000, false_pos:21393, acc:0.6177,ppv:0.6111,npv:0.6869,auc:0.5488
sell -> true_pos:2486.0000, false_pos:2761, acc:0.6640,ppv:0.4738,npv:0.6821,auc:0.5280

count: 95
TESTING: min_change_EMA26_pos_val,Close
buy count: 0
hold count: 54456
sell count: 5803
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33066.0000, false_pos:21390, acc:0.6086,ppv:0.6072,npv:0.6216,auc:0.5410
sell -> true_pos:2439.0000, false_pos:3364, acc:0.6532,ppv:0.4203,npv:0.6780,auc:0.5193

count: 96
TESTING: min_change_EMA26_neg_val,Close
buy count: 0
hold count: 54598
sell count: 5661
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos

...

buy count: 0
hold count: 57112
sell count: 3147
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:34701.0000, false_pos:22411, acc:0.6188,ppv:0.6076,npv:0.8217,auc:0.5438
sell -> true_pos:2129.0000, false_pos:1018, acc:0.6870,ppv:0.6765,npv:0.6875,auc:0.5407

count: 1249
TESTING: min_change_close_neg_val,max_change_MACD_neg_val
buy count: 0
hold count: 56461
sell count: 3798
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:34425.0000, false_pos:22036, acc:0.6204,ppv:0.6097,npv:0.7796,auc:0.5474
sell -> true_pos:2331.0000, false_pos:1467, acc:0.6829,ppv:0.6137,npv:0.6875,auc:0.5401

count: 1250
TESTING: min_change_close_neg_val,min_change_MACD_pos_val
buy count: 0
hold count: 57065
sell count: 3194
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:34677.0000, false_pos:22388, acc:0.6188,ppv:0.6077,npv:0.8168,auc:0.5439
sell -> true_pos:2151.000

buy count: 0
hold count: 56575
sell count: 3684
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33978.0000, false_pos:22597, acc:0.6037,ppv:0.6006,npv:0.6515,auc:0.5298
sell -> true_pos:1709.0000, false_pos:1975, acc:0.6641,ppv:0.4639,npv:0.6772,auc:0.5183

count: 1271
TESTING: min_change_MACD_pos_val,max_change_MACD_neg_val
buy count: 0
hold count: 56176
sell count: 4083
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33707.0000, false_pos:22469, acc:0.6013,ppv:0.6000,npv:0.6192,auc:0.5285
sell -> true_pos:1708.0000, false_pos:2375, acc:0.6575,ppv:0.4183,npv:0.6748,auc:0.5133

count: 1272
TESTING: min_change_MACD_neg_val,max_change_MACD_neg_val
buy count: 0
hold count: 56592
sell count: 3667
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33980.0000, false_pos:22612, acc:0.6035,ppv:0.6004,npv:0.6504,auc:0.5295
sell -> true_pos:1746.0000,

buy count: 0
hold count: 57432
sell count: 2827
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:34329.0000, false_pos:23103, acc:0.6011,ppv:0.5977,npv:0.6700,auc:0.5247
sell -> true_pos:1467.0000, false_pos:1360, acc:0.6703,ppv:0.5189,npv:0.6778,auc:0.5198

count: 1293
TESTING: min_change_MACD_neg_val,min_change_EMA26_neg_val
buy count: 0
hold count: 52864
sell count: 7395
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33065.0000, false_pos:19799, acc:0.6350,ppv:0.6255,npv:0.7029,auc:0.5728
sell -> true_pos:3804.0000, false_pos:3591, acc:0.6721,ppv:0.5144,npv:0.6941,auc:0.5507

count: 1294
TESTING: min_change_MACD_neg_val,max_change_signal_pos_val
buy count: 0
hold count: 57248
sell count: 3011
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:34214.0000, false_pos:23034, acc:0.6004,ppv:0.5976,npv:0.6519,auc:0.5244
sell -> true_pos:1509.00

buy count: 0
hold count: 60259
sell count: 0
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:35262.0000, false_pos:24997, acc:0.5852,ppv:0.5852,npv:0.0000,auc:0.5000
sell -> true_pos:0.0000, false_pos:0, acc:0.6685,ppv:0.0000,npv:0.6685,auc:0.5000

count: 1315
TESTING: min_change_signal_neg_val,min_change_EMA26_pos_val
buy count: 0
hold count: 56527
sell count: 3732
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33985.0000, false_pos:22542, acc:0.6047,ppv:0.6012,npv:0.6578,auc:0.5310
sell -> true_pos:1765.0000, false_pos:1967, acc:0.6652,ppv:0.4729,npv:0.6779,auc:0.5198

count: 1316
TESTING: min_change_EMA26_neg_val,max_change_signal_pos_val
buy count: 0
hold count: 54980
sell count: 5279
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:33812.0000, false_pos:21168, acc:0.6247,ppv:0.6150,npv:0.7253,auc:0.5560
sell -> true_pos:2729.0000, fa

,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,ppv,npv,auc
0,"[Open, Close]",2,buy,3,7,55229,5020,0.000597,0.000127,0.916577,0.300000,0.916679,0.500235
1,"[Open, Close]",2,hold,33011,20702,4295,2251,0.936164,0.828179,0.619094,0.614581,0.656126,0.553992
2,"[Open, Close]",2,sell,2896,3640,36645,17078,0.144988,0.090356,0.656184,0.443084,0.682110,0.527316
0,"[Open, High]",2,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000
1,"[Open, High]",2,hold,33763,22300,2697,1499,0.957490,0.892107,0.605055,0.602233,0.642755,0.532691
2,"[Open, High]",2,sell,1791,2405,37880,18183,0.089667,0.059700,0.658341,0.426835,0.675668,0.514983
0,"[Open, Low]",2,buy,6,32,55204,5017,0.001195,0.000579,0.916212,0.157895,0.916690,0.500308
1,"[Open, Low]",2,hold,32984,20917,4080,2278,0.935398,0.836780,0.615078,0.611937,0.641711,0.549309
2,"[Open, Low]",2,sell,2684,3636,36649,17290,0.134375,0.090257,0.652732,0.424684,0.679453,0.522059
0,"[Volume, Open]",2,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000


In [17]:
# tpr_pair_df["ppv"] = tpr_pair_df["true_pos"] / sum(["true_pos","false_pos"])
tpr_pair_df.loc[(tpr_pair_df["signal"] == "buy") & (tpr_pair_df["true_pos"] > 0),:].sort_values("auc",ascending=False)

,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,ppv,npv,auc
0,"[min_change_close_neg_val, max_change_close_ne...",2,buy,639,889,54347,4384,0.127215,0.016095,0.912494,0.418194,0.925355,0.555560
0,"[prev_min_close, max_change_close_neg_val]",2,buy,419,592,54644,4604,0.083416,0.010718,0.913772,0.414441,0.922293,0.536349
0,"[min_change_MACD_neg_val, max_change_close_neg...",2,buy,343,597,54639,4680,0.068286,0.010808,0.912428,0.364894,0.921105,0.528739
0,"[prev_min_close_date_change, max_change_close_...",2,buy,317,336,54900,4706,0.063110,0.006083,0.916328,0.485452,0.921048,0.528513
0,"[max_change_close_neg_val, Low]",2,buy,308,586,54650,4715,0.061318,0.010609,0.912030,0.344519,0.920576,0.525354
0,"[max_change_close_neg_val, Open]",2,buy,306,571,54665,4717,0.060920,0.010337,0.912245,0.348917,0.920565,0.525291
0,"[max_change_close_neg_val, close_shift1]",2,buy,304,567,54669,4719,0.060522,0.010265,0.912279,0.349024,0.920540,0.525128
0,"[max_change_close_neg_val, Close]",2,buy,300,575,54661,4723,0.059725,0.010410,0.912080,0.342857,0.920467,0.524658
0,"[max_change_close_neg_val, High]",2,buy,292,570,54666,4731,0.058133,0.010319,0.912030,0.338747,0.920350,0.523907
0,"[prev_max_close_date_change, max_change_close_...",2,buy,284,557,54679,4739,0.056540,0.010084,0.912113,0.337693,0.920243,0.523228


# Cumulative feature testing

In [18]:
#Put the features into an order
tpr_pair_df = tpr_pair_df.sort_values("auc",ascending=False).reset_index(drop=True)
ft_li = tpr_pair_df.iloc[0]['features'][:2] #Put in the first row to begin
count = 0
#Loop through the rows, each time a new item is added rest and find the rows which contain that feature
while len(ft_li) < len(feature_cols):
    print('\nNEW LOOP')
    print(len(ft_li))
    count += 1
    #filter for rows with just the later of the two features
    tmp_df = tpr_pair_df[[ft_pair[-1] in x for x in tpr_pair_df['features']]]
    ft_found = False
    #Loop through these rows and put in the next item with the best relationship with this feature
    for index,row in tmp_df.iterrows():
        if row.features[0] not in ft_li:
            ft_li.append(row.features[0])
            print('APPEND {}'.format(row.features[0]))
            ft_found = True
        elif row.features[1] not in ft_li:
            ft_li.append(row.features[1])
            print('APPEND {}'.format(row.features[1]))
            ft_found = True
        if ft_found != False:
            break
    if count > 50:
        print('EMERGENCY BREA')
        break
ft_li


NEW LOOP
2
APPEND min_change_signal_neg_val

NEW LOOP
3
APPEND close_26_norm

NEW LOOP
4
APPEND min_change_close_pos_val

NEW LOOP
5
APPEND change_EMA26_shift1

NEW LOOP
6
APPEND max_change_close_neg_val

NEW LOOP
7
APPEND min_change_EMA26_neg_val

NEW LOOP
8
APPEND Close

NEW LOOP
9
APPEND MACD_neg_val

NEW LOOP
10
APPEND Low

NEW LOOP
11
APPEND signal_neg_val

NEW LOOP
12
APPEND min_change_close_neg_val

NEW LOOP
13
APPEND per_change_price_neg_val

NEW LOOP
14
APPEND Open

NEW LOOP
15
APPEND close_shift1

NEW LOOP
16
APPEND min_change_MACD_neg_val

NEW LOOP
17
APPEND max_change_MACD_neg_val

NEW LOOP
18
APPEND High

NEW LOOP
19
APPEND change_price_neg_val

NEW LOOP
20
APPEND per_change_price

NEW LOOP
21
APPEND MACD_pos_val

NEW LOOP
22
APPEND change_close_shift1

NEW LOOP
23
APPEND max_change_EMA26_neg_val

NEW LOOP
24
APPEND prev_min_signal_date_change

NEW LOOP
25
APPEND signal_pos_val

NEW LOOP
26
APPEND change_price

NEW LOOP
27
APPEND prev_min_EMA26_date_change

NEW LOOP
28
AP

['close_52_norm',
 'close_13_norm',
 'min_change_signal_neg_val',
 'close_26_norm',
 'min_change_close_pos_val',
 'change_EMA26_shift1',
 'max_change_close_neg_val',
 'min_change_EMA26_neg_val',
 'Close',
 'MACD_neg_val',
 'Low',
 'signal_neg_val',
 'min_change_close_neg_val',
 'per_change_price_neg_val',
 'Open',
 'close_shift1',
 'min_change_MACD_neg_val',
 'max_change_MACD_neg_val',
 'High',
 'change_price_neg_val',
 'per_change_price',
 'MACD_pos_val',
 'change_close_shift1',
 'max_change_EMA26_neg_val',
 'prev_min_signal_date_change',
 'signal_pos_val',
 'change_price',
 'prev_min_EMA26_date_change',
 'EMA26_shift1',
 'max_change_EMA26_pos_val',
 'prev_min_close',
 'prev_max_close',
 'max_change_close_pos_val',
 'prev_max_EMA26_date_change',
 'prev_min_MACD_date_change',
 'min_change_MACD_pos_val',
 'min_change_EMA26_pos_val',
 'prev_max_signal_date_change',
 'change_price_pos_val',
 'max_change_signal_pos_val',
 'min_change_signal_pos_val',
 'Volume',
 'vol_shift1',
 'max_change_

In [19]:
#Itterate through the rows adding a new feature each time and add to the dataframe of results
tpr_cum_df = pd.DataFrame([])
count = 0
print("TOTAL TO RUN: {}".format(len(ft_li)))
cum_ft_li = []
for ft in ft_li:
    print("")
    print("count: {}".format(count))
    count += 1
    cum_ft_li.append(ft)
    print("TESTING: {}".format(cum_ft_li))
    #Create a list of unique values
    unique_classes = df_model_rand["signal"].unique()
#     print("unique_classes:",unique_classes)
    #Remove rows with nan or inf
    tmp_li = cum_ft_li.copy()
    tmp_li.append("signal")
    tmp_df_train = df_train[tmp_li].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    tmp_df_test = df_test[tmp_li].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
    #Build each model and add to the models dictionary
    models = build_models(unique_classes,tmp_df_train,cum_ft_li)
#     print(models)
    #Calculate the probabilities of each
    modelled_probs = calc_probs_df(models,unique_classes,tmp_df_test,cum_ft_li)
#     print("modelled_probs: {}".format(modelled_probs.head()))
    #Compare these results to the actual results
    modelled_probs["real_signal"] = tmp_df_test["signal"]
    #Calculate the True Positive Rate (TPR) for each of the unique classifiers
    tpr_cum_df = tpr_cum_df.append(calc_tpr(modelled_probs["signal"],modelled_probs["real_signal"],cum_ft_li,unique_classes))
tpr_cum_df

TOTAL TO RUN: 52

count: 0
TESTING: ['close_52_norm']
buy count: 0
hold count: 44946
sell count: 15313
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:30401.0000, false_pos:14545, acc:0.6780,ppv:0.6764,npv:0.6826,auc:0.6401
sell -> true_pos:7366.0000, false_pos:7947, acc:0.6589,ppv:0.4810,npv:0.7195,auc:0.5858

count: 1
TESTING: ['close_52_norm', 'close_13_norm']
buy count: 0
hold count: 42628
sell count: 17631
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:30004.0000, false_pos:12624, acc:0.7032,ppv:0.7039,npv:0.7018,auc:0.6729
sell -> true_pos:8529.0000, false_pos:9102, acc:0.6590,ppv:0.4838,npv:0.7315,auc:0.6005

count: 2
TESTING: ['close_52_norm', 'close_13_norm', 'min_change_signal_neg_val']
buy count: 0
hold count: 42633
sell count: 17626
buy -> true_pos:0.0000, false_pos:0, acc:0.9166,ppv:0.0000,npv:0.9166,auc:0.5000
hold -> true_pos:29994.0000, false_pos:12639, acc:0.7028,ppv:0.

buy count: 1948
hold count: 43501
sell count: 14810
buy -> true_pos:874.0000, false_pos:1074, acc:0.9133,ppv:0.4487,npv:0.9288,auc:0.5773
hold -> true_pos:30439.0000, false_pos:13062, acc:0.7032,ppv:0.6997,npv:0.7122,auc:0.6703
sell -> true_pos:7497.0000, false_pos:7313, acc:0.6716,ppv:0.5062,npv:0.7255,auc:0.5969

count: 17
TESTING: ['close_52_norm', 'close_13_norm', 'min_change_signal_neg_val', 'close_26_norm', 'min_change_close_pos_val', 'change_EMA26_shift1', 'max_change_close_neg_val', 'min_change_EMA26_neg_val', 'Close', 'MACD_neg_val', 'Low', 'signal_neg_val', 'min_change_close_neg_val', 'per_change_price_neg_val', 'Open', 'close_shift1', 'min_change_MACD_neg_val', 'max_change_MACD_neg_val']
buy count: 1966
hold count: 43511
sell count: 14782
buy -> true_pos:882.0000, false_pos:1084, acc:0.9133,ppv:0.4486,npv:0.9290,auc:0.5780
hold -> true_pos:30449.0000, false_pos:13062, acc:0.7034,ppv:0.6998,npv:0.7126,auc:0.6705
sell -> true_pos:7480.0000, false_pos:7302, acc:0.6715,ppv:0.506

buy count: 1919
hold count: 43667
sell count: 14673
buy -> true_pos:864.0000, false_pos:1055, acc:0.9135,ppv:0.4502,npv:0.9287,auc:0.5765
hold -> true_pos:30534.0000, false_pos:13133, acc:0.7036,ppv:0.6992,npv:0.7150,auc:0.6703
sell -> true_pos:7490.0000, false_pos:7183, acc:0.6736,ppv:0.5105,npv:0.7261,auc:0.5983

count: 28
TESTING: ['close_52_norm', 'close_13_norm', 'min_change_signal_neg_val', 'close_26_norm', 'min_change_close_pos_val', 'change_EMA26_shift1', 'max_change_close_neg_val', 'min_change_EMA26_neg_val', 'Close', 'MACD_neg_val', 'Low', 'signal_neg_val', 'min_change_close_neg_val', 'per_change_price_neg_val', 'Open', 'close_shift1', 'min_change_MACD_neg_val', 'max_change_MACD_neg_val', 'High', 'change_price_neg_val', 'per_change_price', 'MACD_pos_val', 'change_close_shift1', 'max_change_EMA26_neg_val', 'prev_min_signal_date_change', 'signal_pos_val', 'change_price', 'prev_min_EMA26_date_change', 'EMA26_shift1']
buy count: 1922
hold count: 43684
sell count: 14653
buy -> tru

hold -> true_pos:30418.0000, false_pos:12643, acc:0.7098,ppv:0.7064,npv:0.7183,auc:0.6784
sell -> true_pos:8026.0000, false_pos:7011, acc:0.6854,ppv:0.5338,npv:0.7358,auc:0.6139

count: 36
TESTING: ['close_52_norm', 'close_13_norm', 'min_change_signal_neg_val', 'close_26_norm', 'min_change_close_pos_val', 'change_EMA26_shift1', 'max_change_close_neg_val', 'min_change_EMA26_neg_val', 'Close', 'MACD_neg_val', 'Low', 'signal_neg_val', 'min_change_close_neg_val', 'per_change_price_neg_val', 'Open', 'close_shift1', 'min_change_MACD_neg_val', 'max_change_MACD_neg_val', 'High', 'change_price_neg_val', 'per_change_price', 'MACD_pos_val', 'change_close_shift1', 'max_change_EMA26_neg_val', 'prev_min_signal_date_change', 'signal_pos_val', 'change_price', 'prev_min_EMA26_date_change', 'EMA26_shift1', 'max_change_EMA26_pos_val', 'prev_min_close', 'prev_max_close', 'max_change_close_pos_val', 'prev_max_EMA26_date_change', 'prev_min_MACD_date_change', 'min_change_MACD_pos_val', 'min_change_EMA26_pos_

buy count: 2148
hold count: 42744
sell count: 15367
buy -> true_pos:1032.0000, false_pos:1116, acc:0.9152,ppv:0.4804,npv:0.9313,auc:0.5926
hold -> true_pos:30370.0000, false_pos:12374, acc:0.7135,ppv:0.7105,npv:0.7207,auc:0.6831
sell -> true_pos:8261.0000, false_pos:7106, acc:0.6877,ppv:0.5376,npv:0.7391,auc:0.6186

count: 43
TESTING: ['close_52_norm', 'close_13_norm', 'min_change_signal_neg_val', 'close_26_norm', 'min_change_close_pos_val', 'change_EMA26_shift1', 'max_change_close_neg_val', 'min_change_EMA26_neg_val', 'Close', 'MACD_neg_val', 'Low', 'signal_neg_val', 'min_change_close_neg_val', 'per_change_price_neg_val', 'Open', 'close_shift1', 'min_change_MACD_neg_val', 'max_change_MACD_neg_val', 'High', 'change_price_neg_val', 'per_change_price', 'MACD_pos_val', 'change_close_shift1', 'max_change_EMA26_neg_val', 'prev_min_signal_date_change', 'signal_pos_val', 'change_price', 'prev_min_EMA26_date_change', 'EMA26_shift1', 'max_change_EMA26_pos_val', 'prev_min_close', 'prev_max_close

buy count: 2172
hold count: 42672
sell count: 15415
buy -> true_pos:1040.0000, false_pos:1132, acc:0.9151,ppv:0.4788,npv:0.9314,auc:0.5933
hold -> true_pos:30315.0000, false_pos:12357, acc:0.7128,ppv:0.7104,npv:0.7187,auc:0.6827
sell -> true_pos:8292.0000, false_pos:7123, acc:0.6879,ppv:0.5379,npv:0.7395,auc:0.6192

count: 49
TESTING: ['close_52_norm', 'close_13_norm', 'min_change_signal_neg_val', 'close_26_norm', 'min_change_close_pos_val', 'change_EMA26_shift1', 'max_change_close_neg_val', 'min_change_EMA26_neg_val', 'Close', 'MACD_neg_val', 'Low', 'signal_neg_val', 'min_change_close_neg_val', 'per_change_price_neg_val', 'Open', 'close_shift1', 'min_change_MACD_neg_val', 'max_change_MACD_neg_val', 'High', 'change_price_neg_val', 'per_change_price', 'MACD_pos_val', 'change_close_shift1', 'max_change_EMA26_neg_val', 'prev_min_signal_date_change', 'signal_pos_val', 'change_price', 'prev_min_EMA26_date_change', 'EMA26_shift1', 'max_change_EMA26_pos_val', 'prev_min_close', 'prev_max_close

,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,ppv,npv,auc
0,[close_52_norm],1,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000
1,[close_52_norm],1,hold,30401,14545,10452,4861,0.862146,0.581870,0.677957,0.676389,0.682557,0.640138
2,[close_52_norm],1,sell,7366,7947,32338,12608,0.368779,0.197269,0.658889,0.481029,0.719486,0.585755
0,"[close_52_norm, close_13_norm]",2,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000
1,"[close_52_norm, close_13_norm]",2,hold,30004,12624,12373,5258,0.850888,0.505021,0.703248,0.703857,0.701775,0.672934
2,"[close_52_norm, close_13_norm]",2,sell,8529,9102,31183,11445,0.427005,0.225940,0.659022,0.483750,0.731514,0.600532
0,"[close_52_norm, close_13_norm, min_change_sign...",3,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000
1,"[close_52_norm, close_13_norm, min_change_sign...",3,hold,29994,12639,12358,5268,0.850604,0.505621,0.702833,0.703540,0.701123,0.672492
2,"[close_52_norm, close_13_norm, min_change_sign...",3,sell,8519,9107,31178,11455,0.426504,0.226064,0.658773,0.483320,0.731311,0.600220
0,"[close_52_norm, close_13_norm, min_change_sign...",4,buy,0,0,55236,5023,0.000000,0.000000,0.916643,0.000000,0.916643,0.500000


In [20]:
tpr_cum_df.loc[(tpr_cum_df["signal"] == "buy"),:].sort_values("auc",ascending=False)

,features,feature_count,signal,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,ppv,npv,auc
0,"[close_52_norm, close_13_norm, min_change_sign...",52,buy,1251,1230,54006,3772,0.249054,0.022268,0.916992,0.504232,0.934716,0.613393
0,"[close_52_norm, close_13_norm, min_change_sign...",51,buy,1186,1195,54041,3837,0.236114,0.021634,0.916494,0.498110,0.933705,0.607240
0,"[close_52_norm, close_13_norm, min_change_sign...",50,buy,1047,1138,54098,3976,0.208441,0.020603,0.915133,0.479176,0.931536,0.593919
0,"[close_52_norm, close_13_norm, min_change_sign...",46,buy,1042,1118,54118,3981,0.207446,0.020240,0.915382,0.482407,0.931479,0.593603
0,"[close_52_norm, close_13_norm, min_change_sign...",48,buy,1043,1133,54103,3980,0.207645,0.020512,0.915150,0.479320,0.931477,0.593566
0,"[close_52_norm, close_13_norm, min_change_sign...",49,buy,1040,1132,54104,3983,0.207048,0.020494,0.915116,0.478821,0.931430,0.593277
0,"[close_52_norm, close_13_norm, min_change_sign...",47,buy,1038,1118,54118,3985,0.206649,0.020240,0.915316,0.481447,0.931415,0.593204
0,"[close_52_norm, close_13_norm, min_change_sign...",44,buy,1037,1118,54118,3986,0.206450,0.020240,0.915299,0.481206,0.931399,0.593105
0,"[close_52_norm, close_13_norm, min_change_sign...",36,buy,1036,1125,54111,3987,0.206251,0.020367,0.915166,0.479408,0.931375,0.592942
0,"[close_52_norm, close_13_norm, min_change_sign...",42,buy,1034,1115,54121,3989,0.205853,0.020186,0.915299,0.481154,0.931354,0.592833


In [21]:
#Convert features into string
tmp_df = tpr_cum_df.copy()
def ft_str(li_in):
    str_out = ''
    for i in li_in:
        str_out += i+','
    str_out = str_out[:-1]
    return str_out
tmp_df["ft_str"] = [ft_str(x) for x in tmp_df["features"]]
tmp_df
#Average the model over all the signals
tpr_cum_sum_df = tmp_df.loc[tmp_df["signal"] == "buy",:][["ft_str","feature_count","true_pos","false_pos","true_neg","false_neg","tpr","fpr","acc","auc"]].groupby("ft_str").mean().reset_index()
tpr_cum_sum_df.sort_values("auc",ascending=False,inplace=True)
tpr_cum_sum_df.reset_index(drop=True)

,ft_str,feature_count,true_pos,false_pos,true_neg,false_neg,tpr,fpr,acc,auc
0,"close_52_norm,close_13_norm,min_change_signal_...",52,1251,1230,54006,3772,0.249054,0.022268,0.916992,0.613393
1,"close_52_norm,close_13_norm,min_change_signal_...",51,1186,1195,54041,3837,0.236114,0.021634,0.916494,0.607240
2,"close_52_norm,close_13_norm,min_change_signal_...",50,1047,1138,54098,3976,0.208441,0.020603,0.915133,0.593919
3,"close_52_norm,close_13_norm,min_change_signal_...",46,1042,1118,54118,3981,0.207446,0.020240,0.915382,0.593603
4,"close_52_norm,close_13_norm,min_change_signal_...",48,1043,1133,54103,3980,0.207645,0.020512,0.915150,0.593566
5,"close_52_norm,close_13_norm,min_change_signal_...",49,1040,1132,54104,3983,0.207048,0.020494,0.915116,0.593277
6,"close_52_norm,close_13_norm,min_change_signal_...",47,1038,1118,54118,3985,0.206649,0.020240,0.915316,0.593204
7,"close_52_norm,close_13_norm,min_change_signal_...",44,1037,1118,54118,3986,0.206450,0.020240,0.915299,0.593105
8,"close_52_norm,close_13_norm,min_change_signal_...",36,1036,1125,54111,3987,0.206251,0.020367,0.915166,0.592942
9,"close_52_norm,close_13_norm,min_change_signal_...",42,1034,1115,54121,3989,0.205853,0.020186,0.915299,0.592833


# Train the final model

In [22]:
#Get the most effective model
fm_features = tpr_cum_sum_df.iloc[0,:]
ft_li = fm_features.ft_str.split(",")
print("BUILDING: {}".format(cum_ft_li))

#Create a list of unique values
unique_classes = df_model_rand["signal"].unique()
print("UNIQUE CLASSES:",unique_classes)
#Remove rows with nan or inf
tmp_li = cum_ft_li.copy()
tmp_li.append("signal")
tmp_df_train = df_train[tmp_li].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
tmp_df_test = df_test[tmp_li].replace([np.inf,-np.inf],np.nan).dropna(axis=0)
#Build each model and add to the models dictionary
final_models = build_models(unique_classes,tmp_df_train,ft_li)
final_models

BUILDING: ['close_52_norm', 'close_13_norm', 'min_change_signal_neg_val', 'close_26_norm', 'min_change_close_pos_val', 'change_EMA26_shift1', 'max_change_close_neg_val', 'min_change_EMA26_neg_val', 'Close', 'MACD_neg_val', 'Low', 'signal_neg_val', 'min_change_close_neg_val', 'per_change_price_neg_val', 'Open', 'close_shift1', 'min_change_MACD_neg_val', 'max_change_MACD_neg_val', 'High', 'change_price_neg_val', 'per_change_price', 'MACD_pos_val', 'change_close_shift1', 'max_change_EMA26_neg_val', 'prev_min_signal_date_change', 'signal_pos_val', 'change_price', 'prev_min_EMA26_date_change', 'EMA26_shift1', 'max_change_EMA26_pos_val', 'prev_min_close', 'prev_max_close', 'max_change_close_pos_val', 'prev_max_EMA26_date_change', 'prev_min_MACD_date_change', 'min_change_MACD_pos_val', 'min_change_EMA26_pos_val', 'prev_max_signal_date_change', 'change_price_pos_val', 'max_change_signal_pos_val', 'min_change_signal_pos_val', 'Volume', 'vol_shift1', 'max_change_signal_neg_val', 'max_change_MACD

{'buy': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'hold': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'sell': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False)}

# Exporting the model

In [23]:
#Import the modules
from sklearn.externals import joblib as jl

In [24]:
#Export the model
jl.dump(final_models,path+r'\log_reg_model.joblib')

['C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\log_reg_model.joblib']

In [25]:
#Export a list of the features for this model
file_object = open(path+r'\log_reg_model_feature_list.txt','w')
file_object.write(fm_features.ft_str)
file_object.close()